<a href="https://colab.research.google.com/github/alexandergribenchenko/Pycaret_Exploration/blob/main/Acamica_PY_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proyecto 04 - Informe final de carrera**

### (Acámica - DS online 40 - Proyecto 04 - Alexander Ortega)

**Nota introductoria para la ejecución del notebook:** El presente notebook fue desarrollado empleando Google Colab y se recomienda, con el fin de obtener una correcta visualización de la totalidad de las gráficas y salidas del notebook, que este sea ejecutado de principio a fin en dicho entorno. Existen algunas gráficas interactivas que el paquete PyCaret genera que no se pueden visualizar en la vista previa del notebook desplegada en el repositorio de github, y que se sólo pueden verse al generar su ejecución.

# **01. Alcance y objeto del proyecto**


Si bien es clara la importancia de haber generado en los anteriores proyectos una implementación gradual (paso a paso) de los modelos de predicción, pues esto nos permitió tener un mayor conocimiento de las implicaciones y beneficios de cada una de las etapas de modelado, este proceso puede tomar un tiempo significativo que en contexto laboral puede dificultar su implementación. 

Es por ello que en este último proyecto nuestra finalidad es explorar una herramienta que nos permita generar de manera automatizada evaluaciones para un conjunto amplio de modelos, conociendo con ello de manera ágil los resultados para algunos caminos exploratorios, que después si así lo quisiéramos podríamos afinar. Así pues, la finalidad del presente proyecto no es mejorar de antemano las métricas obtenidas para un modelo en específico ni describir en detalle su funcionamiento,  sino explorar una herramienta que nos permita hacer de manera automatizada un conjunto de pruebas analíticas mucho más amplio y en un menor tiempo.

La librería empleada para hacer dicha automatización es [PyCaret](https://pycaret.org/) y el contexto del problema analitico bajo el cual se explorará la herramienta será el abordado en el Proyecto_02 que consistió en la evaluación de un conjunto de modelos de predicción de precios para inmuebles de la Ciudad de Buenos Aires entre 2019 y enero de 2020.

El actual proyecto abordará de manera conjunta a partir de la librería Pycaret no únicamente el problema de la generación de modelos de regresión de aprendizaje supervisado para la estimación de precio de los inmuebles, sino que también complementaremos su alcance con la evaluación de modelos de clustereizacion ampliando así además los resultados de los anteriores proyectos. 


# **02. Presentación de la herramienta central del proyecto (PyCaret)**

Tal como está descrito en su [página oficial](https://pycaret.org/guide/), PyCaret es una librería de aprendizaje automático de código abierto en Python que tiene como objetivo reducir el tiempo empleado desde la formulación de las hipótesis hasta la obtención de resultados en diversos contextos analiticos. 

Si bien la herramienta se sugiere como de uso para todo tipo de público, enfatizamos de manera particular en la recomendación que generan para científicos de datos experimentados que desean aumentar la productividad en los experimentos de aprendizaje automático mediante el uso de PyCaret en sus flujos de trabajo, dado que conocen el contexto de la herramienta que y pueden evaluar y enfocar con mayor criterio los resultados y funcionalidades que las herramienta entrega. 

Es importante destacar también que la implementación presentada en este proyecto tomó como referentes los  [tutoriales oficiales](https://pycaret.readthedocs.io/en/latest/tutorials.html#regression) para problemas de regresión y clusterización que PyCaret comparte en su portal.

# **03.Instalación y carga de las librerías a emplear**

In [1]:
#!pip install --upgrade scikit-learn

In [2]:
!pip install --upgrade xgboost

Requirement already up-to-date: xgboost in /usr/local/lib/python3.7/dist-packages (1.4.2)


In [3]:
!pip install --upgrade catboost

Requirement already up-to-date: catboost in /usr/local/lib/python3.7/dist-packages (0.26)


In [4]:
!pip install --upgrade numpy

  Using cached https://files.pythonhosted.org/packages/3f/03/c3526fb4e79a793498829ca570f2f868204ad9a8040afcd72d82a8f121db/numpy-1.21.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
ERROR: yellowbrick 1.3.post1 has requirement numpy<1.20,>=1.16.0, but you'll have numpy 1.21.0 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.21.0 which is incompatible.
ERROR: pyldavis 3.3.1 has requirement pandas>=1.2.0, but you'll have pandas 1.1.5 which is incompatible.
ERROR: pycaret 2.3.2 has requirement numpy==1.19.5, but you'll have numpy 1.21.0 which is incompatible.
ERROR: phik 0.11.2 has requirement scipy>=1.5.2, but you'll have scipy 1.4.1 which is incompatible.
ERROR: pandas-profiling 3.0.0 has requirement requests>=2.24.0, but you'll have requests 2.23.0 which is incompatible.
ERROR: pandas-profiling 3.0.0 has requirement tqdm>=4.48.2, but you'll have tqdm 4.41.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement f

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
import time

In [6]:
!pip install pycaret

  Using cached https://files.pythonhosted.org/packages/08/d6/a6aaa29fea945bc6c61d11f6e0697b325ff7446de5ffd62c2fa02f627048/numpy-1.19.5-cp37-cp37m-manylinux2010_x86_64.whl
ERROR: pyldavis 3.3.1 has requirement numpy>=1.20.0, but you'll have numpy 1.19.5 which is incompatible.
ERROR: pyldavis 3.3.1 has requirement pandas>=1.2.0, but you'll have pandas 1.1.5 which is incompatible.
ERROR: phik 0.11.2 has requirement scipy>=1.5.2, but you'll have scipy 1.4.1 which is incompatible.
ERROR: pandas-profiling 3.0.0 has requirement requests>=2.24.0, but you'll have requests 2.23.0 which is incompatible.
ERROR: pandas-profiling 3.0.0 has requirement tqdm>=4.48.2, but you'll have tqdm 4.41.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.21.0
    Uninstalling 

In [7]:
from pycaret.utils import enable_colab
enable_colab()

from pycaret.utils import version
version()

Colab mode enabled.


'2.3.2'

# **04.Preparación del Dataset de Análisis**

El dataset base que emplearemos para explorar las herramientas de la libreria PyCaret tanto en los problemas de regresión como el los de clustering corresponde con un dataset ‘curado’ que hemos obtenido del [Proyecto_02](https://github.com/alexandergribenchenko/Acamica_Proyecto_02/blob/main/DS_Proyecto_02_Entregado.ipynb). Aunque la finalidad del presente proyecto no es obtener de antemano mejores resultados en las métricas de los modelos que los presentados en el proyecto descrito, si resulta útil que pueda existir una secuencia comparativa en el desarrollo de los modelos a partir de un set de datos similar pues esto nos permitirá tener una idea de los resultados que se pueden tener con una implementación automatizada. 

En este proyecto no nos detendremos en un detallado análisis exploratorio de datos para describir el origen del dataset, que puede visualizarse en detalle en el [Proyecto_02](https://github.com/alexandergribenchenko/Acamica_Proyecto_02/blob/main/DS_Proyecto_02_Entregado.ipynb). Simplemente destacamos que es un dataset que en el que se ha hecho de antemano eliminación de outliers bajo criterios de análisis específicos, y que cuenta con los mismos registros tanto en cantidad como en índice que los empleados para entrenar y generar los modelos en el Proyecto_02.  Sin embargo no se han desarrollado sobre el transformaciones de escalado, imputación o encoding, pues la idea es explotar las funcionalidades de PyCaret en este aspecto. 

En detalle el dataset curado obtenido como parte del proyecto_02, y que emplearemos  en las etapas de regresión y clustering con PyCaret es el siguiente:

In [8]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [9]:
path_data = '/content/drive/MyDrive/Colab Notebooks/00_Acamica/PY_04_Acamica/'

In [10]:
df = pd.read_csv(path_data+'df_base_PY_04.csv', sep =',')
df

,l3,property_type,lat,lon,rooms,bedrooms,bathrooms,surface_total,surface_covered,price
0,Boedo,PH,-34.624056,-58.412110,2.0,1.0,2.0,70.0,58.0,159000.0
1,Palermo,PH,-34.593569,-58.427474,2.0,1.0,1.0,45.0,45.0,125000.0
2,Palermo,PH,-34.581294,-58.436754,2.0,1.0,1.0,85.0,50.0,295000.0
3,Villa Crespo,PH,-34.593779,-58.441239,2.0,1.0,1.0,56.0,56.0,150000.0
4,Villa Crespo,PH,-34.600988,-58.433061,2.0,1.0,1.0,70.0,70.0,159500.0
...,...,...,...,...,...,...,...,...,...,...
67863,Palermo,Departamento,-34.587152,-58.437724,3.0,2.0,2.0,120.0,82.0,430000.0
67864,Palermo,Departamento,-34.581425,-58.442707,3.0,2.0,2.0,88.0,82.0,399000.0
67865,Palermo,PH,-34.576573,-58.434394,3.0,2.0,1.0,84.0,84.0,178000.0
67866,Palermo,PH,-34.585355,-58.427194,3.0,2.0,2.0,144.0,134.0,480000.0


Con el fin de contar con un set de entrenamiento y uno de prueba generamos de manera manual un split de los datos. No será necesario hacer uso del train-test-split de ScikitLearn pues con PyCaret podremos trabajar en un mismo dataset con los features de entrada y con el target como veremos más adelante. Así, los sets de entrenamiento y prueba serán los siguientes:

In [11]:
data = df.sample(frac=0.7, random_state=786)
data_unseen = df.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print('Cantidad de muestras para el entrenamiento: ' + str(data.shape))
print('Cantidad de muestras para el test: ' + str(data_unseen.shape))

Cantidad de muestras para el entrenamiento: (47508, 10)
Cantidad de muestras para el test: (20360, 10)


# **05.Implementación de PyCaret en el contexto del aprendizaje supervisado (Regresión)**

## **05.01. Configurando el ambiente de trabajo en PyCaret (Regresión)**

La función `setup()` es la principal función de PyCaret y nos permite inicializar el entorno de trabajo creando un pipeline de transformación para preparar los datos para el modelado. Es por ello que debe se debe ejecutar antes que cualquier otra función al emplear PyCaret. Para hacer uso de la función `setup()` cargaremos de manera inicial la totalidad de las funciones y métodos con que cuenta la sección de regresión de la librería:

In [12]:
from pycaret.regression import *

La función `setup()` presenta mas de 50 parámetros que se pueden personalizar acorde a todos los tipos de transformación que se desean automatizar como parte del pipeline, sin embargo solamente dos de ellos son obligatorios: el dataframe de pandas que contiene la información  y la definición de la variable de target. En nuestro caso específico definiremos únicamente de manera adicional la inclusión de la característica numérica de la variable bathrooms que no estaba siendo reconocida de manera adecuada en la automatización. Los elementos restates PyCaret los tomará con sus valores asignados por defecto. Veamos pues los elementos que presenta PyCaret como posibilidad a ser incluidos como parte de su Pipeline:

In [15]:
pipeline = setup(data = data, target = 'price', numeric_features=['bathrooms'])

,Description,Value
0,session_id,1838
1,Target,price
2,Original Data,"(47508, 10)"
3,Missing Values,True
4,Numeric Features,7
5,Categorical Features,2
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(33255, 66)"


Vemos que aunque muchos de los parámetros aparezcan por defecto con la configuración de False o None, podrían activarse y configurarse como parte del pipeline, destacando entre ellos las transformaciones de escalado, imputación, y encoding ya tratados en el Proyecto_02. Sin embargo con el fin de mantener el uso más simple de la librería emplearemos las restantes variables en sus valores por defecto, y evaluaremos qué resultados obtendremos bajo tal configuración.



## **05.02. Evaluando modelos en simultaneo (Regresión)**

Aunque la automatización del pipeline es ya de antemano una ventaja significativa, no es aún la etapa que mayor valor puede aportar en la automatización del modelado a través de PyCaret. La etapa que mayor valor puede aportar a mi criterio, es la automatización de la evaluación de un muy amplio conjunto de modelos respecto a las métricas más significativas que este problema presenta. En específico para los problemas de regresión PyCaret se pueden evaluar de manera automatizada 25 tipos de modelos de regresión diferentes respecto a sus métricas más comunes (MAE, MSE, RMSE, R2, RMSLE y MAPE ). A continuación se presentan los 25 modelos descritos para el caso de problemas de regresión:

In [16]:
models()

,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model._base.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model._coordinate_descent.Lasso,True
ridge,Ridge Regression,sklearn.linear_model._ridge.Ridge,True
en,Elastic Net,sklearn.linear_model._coordinate_descent.Elast...,True
lar,Least Angle Regression,sklearn.linear_model._least_angle.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model._least_angle.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model._omp.OrthogonalMatchingPu...,True
br,Bayesian Ridge,sklearn.linear_model._bayes.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model._bayes.ARDRegression,False


La función que permite hacer la comparación de los modelos se denomina `compare_models()`. De la misma manera que en la sección anterior, esta función cuenta con gran cantidad de parámetros, sin embargo en nuestro caso específico únicamente definiremos 2 y emplearemos las restantes configuraciones por defecto con el fin de mantener el uso más simple de la librería y evaluar qué resultados obtendremos bajo tal configuración. Los 2 parámetros que configuraremos son la cantidad de folds con que se evaluarán las métricas (llevándolas a 5 y no a 10 para optimizar tiempo) y la cantidad de modelos que se almacenarán tras la ejecución al hacer la selección del top 5 respecto a la métrica R2 (que es la métrica por defecto empleada en el caso de los problemas de regresión). Veamos los resultados para los modelos evaluados:

In [17]:
start_time = time.time()
best = compare_models(fold = 5 , n_select=5)
print("--- Tiempo de ejecución en minutos ---:", round((time.time() - start_time)/60,2))

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,1.729190e+04,8.767811e+08,2.960503e+04,8.968000e-01,0.1535,1.006000e-01,14.202
et,Extra Trees Regressor,1.654312e+04,8.802703e+08,2.966770e+04,8.964000e-01,0.1533,9.640000e-02,15.488
xgboost,Extreme Gradient Boosting,2.256355e+04,1.101125e+09,3.318132e+04,8.704000e-01,0.1735,1.313000e-01,12.998
catboost,CatBoost Regressor,2.359675e+04,1.142432e+09,3.379864e+04,8.655000e-01,0.1768,1.369000e-01,8.366
lightgbm,Light Gradient Boosting Machine,2.499525e+04,1.259821e+09,3.549175e+04,8.517000e-01,0.1856,1.453000e-01,0.422
gbr,Gradient Boosting Regressor,2.861041e+04,1.588642e+09,3.985524e+04,8.130000e-01,0.2061,1.658000e-01,4.604
dt,Decision Tree Regressor,2.126536e+04,1.640303e+09,4.049335e+04,8.069000e-01,0.2034,1.214000e-01,0.266
lr,Linear Regression,3.219407e+04,1.943633e+09,4.408363e+04,7.713000e-01,0.2343,1.884000e-01,0.658
lasso,Lasso Regression,3.218784e+04,1.943695e+09,4.408433e+04,7.712000e-01,0.2335,1.882000e-01,0.898
br,Bayesian Ridge,3.217660e+04,1.944127e+09,4.408910e+04,7.712000e-01,0.2334,1.881000e-01,0.236


--- Tiempo de ejecución en minutos ---: 6.41


Podemos observar como la evaluación de estos más de 20 modelos se logró empleando una única línea, que si bien puede parametrizarse con un mayor grado de detalle, permite tener una imagen macro del resultado de un conjunto de modelos que de manera manuel nos llevaría un importante bloque de código. Esta es a mi criterio la mayor ventaja que presenta la librería, pues en muchos problemas no conocemos de antemano que modelo puede tener un mejor comportamiento y resulta útil una rápida mirada panorámica para saber qué caminos o modelos podemos ir a explorar con mayor detalle.
 
También es de destacar el tiempo de ejecución y los valores de las métricas logradas. Aunque esto por supuesto guarda una estricta dependencia con el tamaño y la calidad de la preparación del dataset, en nuestro caso específico comparado con los resultados del Proyecto_02 tiene valores de métricas bastante buenos (5 modelos top con un R2 superior a 0.8) aun para las configuraciones por defecto empleadas, más aún cuando la implementación y ejecución total del código de más de 20 modelos tomo menos de 5 minutos. Este es sin duda el eje central del actual proyecto y el espíritu de la libreria Pycaret. 

En nuestro caso específico mostramos a continuación los 5 mejores modelos encontrados respecto a la metrica de R2, así como también los hiper parámetros por defecto empleados: 

In [18]:
for model in best:
  print(model)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=1838, verbose=0, warm_start=False)
ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=-1, oob_score=False,
                    random_state=1838, verbose=0, warm_start=False)
XGBRegressor(base_score

## **05.03. Creación de los modelos**

En la presente sección instanciaremos y evaluaremos 3 de los 5 mejores modelos encontrados en la sección anterior a manera de ejemplo para evidenciar el procedimiento a seguir en PyCaret. Esto nos permitirá en secciones posteriores hacer procedimientos específicos con estos modelos ya instalados y evaluados. Es importante destacar que no es mandatorio haber hecho la evaluación de los modelos del numeral anterior para poder instanciar y evaluar los modelos como lo haremos en el presente numeral. La utilidad de desarrollar la evaluación en esta secuencia es que podemos conocer de antemano qué modelos presentan un mejor comportamiento acorde a nuestras necesidades específicas e instanciarlos para hacer procedimientos particulares con estos modelos elegidos.

En esta sección la función a presentar es `create_model()` que es la que permite entrenar y evalúar un modelo empleando validación cruzada a partir del parámetro `fold`. La salida nos permite observar los resultados de las métricas MAE, MSE, RMSE, R2, RMSLE y MAPE para cada valor de fold, así como los estadísticos representativos de las métricas para cada modelo.

Los modelos 3 modelos entre los 5 se eligieron de manera arbitraria con el fin único de ejemplificar su procedimiento, idéntico e independiente del modelo elegido. Los modelos con que presentaremos los ejemplos son: 

- Random Forest (rf)
- Extreme Gradient Boosting (xgboost) 
- Light Gradient Boosting Machine (lightgbm)


### **05.03.01. Random Forest**

In [19]:
model_rf = create_model(best[0], fold=10)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,16759.8072,8.349307e+08,28895.1681,0.9036,0.1562,0.1013
1,16517.7551,7.885813e+08,28081.6906,0.9037,0.1509,0.0993
2,16591.6850,8.113168e+08,28483.6233,0.9030,0.1459,0.0958
3,16984.0473,9.090091e+08,30149.7783,0.8907,0.1526,0.0979
4,16383.3976,7.930510e+08,28161.1614,0.9060,0.1439,0.0936
5,17642.0933,9.178673e+08,30296.3256,0.8946,0.1534,0.1004
6,16569.2692,8.156742e+08,28560.0098,0.9041,0.1483,0.0959
7,17437.0496,9.559770e+08,30918.8776,0.8906,0.1529,0.0983
8,16712.5338,8.195332e+08,28627.4909,0.9078,0.1548,0.1000
9,16663.6281,8.502147e+08,29158.4409,0.8956,0.1473,0.0951


### **05.03.02. XGBoost**

In [20]:
model_xgboost = create_model(best[2])

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,22728.9336,1.085641e+09,32949.0703,0.8747,0.1813,0.1374
1,22290.7812,1.030618e+09,32103.2441,0.8741,0.1737,0.1334
2,22045.1387,1.027595e+09,32056.1230,0.8771,0.1695,0.1291
3,23015.9336,1.149380e+09,33902.5039,0.8618,0.1763,0.1336
4,22889.1133,1.090285e+09,33019.4648,0.8708,0.1706,0.1309
5,22805.7598,1.107972e+09,33286.2188,0.8728,0.1738,0.1311
6,22126.0664,1.048046e+09,32373.5430,0.8767,0.1685,0.1289
7,22733.5215,1.176081e+09,34294.0391,0.8654,0.1712,0.1280
8,22154.3867,1.036194e+09,32189.9727,0.8835,0.1725,0.1310
9,22452.9805,1.081743e+09,32889.8633,0.8672,0.1697,0.1291


### **05.03.03. LightGBM**

In [21]:
model_lightgbm = create_model(best[4])

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,25214.5932,1.255374e+09,35431.2643,0.8551,0.1948,0.1528
1,24557.6731,1.177096e+09,34308.8397,0.8562,0.1864,0.1468
2,24553.9506,1.184850e+09,34421.6497,0.8583,0.1822,0.1436
3,25100.5736,1.324846e+09,36398.4307,0.8407,0.1871,0.1452
4,25066.5150,1.252498e+09,35390.6417,0.8516,0.1826,0.1436
5,25914.5844,1.367899e+09,36985.1165,0.8430,0.1882,0.1471
6,24685.8959,1.221234e+09,34946.1516,0.8564,0.1798,0.1426
7,25249.5205,1.348073e+09,36716.1102,0.8458,0.1850,0.1426
8,24616.7892,1.219719e+09,34924.4717,0.8628,0.1860,0.1452
9,25380.6734,1.296550e+09,36007.6315,0.8408,0.1830,0.1441


In [27]:
model_lightgbm.get_params

<bound method LGBMModel.get_params of LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=1838, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)>

## **05.04. Optimización de hiperparámetros**

Otra de las etapas en las que resulta útil tener un procedimiento automatizado asistido que nos permita mejorar nuestros modelos es la optimización de hiperparámetros. Al instanciar los modelos con la función `create_model` los hiperparámetros  que se emplean son los que  el modelo tiene por defecto, que pueden ser mejorados a partir de búsquedas sobre un conjunto de hiperparámetros más amplio.

PyCaret emplea para esta labor específica la función `tune_model`. que se encarga de ajustar automáticamente los hiperparámetros de un modelo utilizando `Random Grid Search`, en un espacio de muestreo predefinido. La salida imprime una cuadrícula de puntuación que muestra MAE, MSE, RMSE, R2, RMSLE y MAPE para cada fold. Sin embargo existen diversos parámetros de que se pueden personalizar como parte de la función  `tune_model` de los cuales exploraremos 3: 
- La posibilidad de elegir la métrica respecto a la cual se hará la optimización. 
- La selección de la cantidad de iteraciones que desean evaluarse en el espacio muestral.
- La personalización del espacio muestral de hiperparametros sobre el que queremos que se haga la optimización.

Elegiremos de manera arbitraria el modelo Light Gradient Boosting Machine (lightgbm) para implementar el procedimiento de optimización. El procedimiento es completamente análogo para cualquier otro tipo de modelo. Solo se debe tener en cuenta que si se desea personalizar el espacio muestral, este debe presentar hiperparametros propios del modelo a optimizar.


### **05.04.01. Optimizacion por defecto (espacio muestral predefinido y configuraciones por defecto)**

In [22]:
tuned_lightgbm_01 = tune_model(model_lightgbm)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,19839.4477,9.452757e+08,30745.3363,0.8909,0.1722,0.1231
1,19434.1905,8.829590e+08,29714.6263,0.8921,0.1624,0.1173
2,19470.2193,8.879366e+08,29798.2643,0.8938,0.1562,0.1139
3,20093.5593,1.019706e+09,31932.8403,0.8774,0.1630,0.1169
4,19977.7232,9.562823e+08,30923.8137,0.8867,0.1606,0.1157
5,20445.4698,1.001615e+09,31648.3091,0.8850,0.1639,0.1179
6,19129.3217,8.838949e+08,29730.3697,0.8960,0.1553,0.1126
7,20188.7622,1.031394e+09,32115.3280,0.8820,0.1620,0.1150
8,19684.3904,9.288085e+08,30476.3596,0.8955,0.1624,0.1165
9,19582.6227,9.371451e+08,30612.8250,0.8849,0.1578,0.1132


In [26]:
tuned_lightgbm_01.get_params

<bound method LGBMModel.get_params of LGBMRegressor(bagging_fraction=1.0, bagging_freq=5, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
              importance_type='split', learning_rate=0.2, max_depth=-1,
              min_child_samples=26, min_child_weight=0.001, min_split_gain=0.2,
              n_estimators=50, n_jobs=-1, num_leaves=256, objective=None,
              random_state=1838, reg_alpha=1, reg_lambda=4, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)>

### **05.04.02. Optimizacion con selección del número de iteraciones y de la métrica a optimizar**

In [30]:
tuned_lightgbm_02 = tune_model(model_lightgbm,n_iter = 50, optimize = 'R2')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,18097.1308,9.344074e+08,30568.0782,0.8922,0.1672,0.1092
1,17731.2089,8.563501e+08,29263.4601,0.8954,0.1596,0.1066
2,17333.8244,8.215802e+08,28663.2206,0.9017,0.1486,0.1007
3,18357.5413,9.613555e+08,31005.7327,0.8844,0.1595,0.1066
4,18061.1866,9.088856e+08,30147.7297,0.8923,0.1549,0.1034
5,18365.9953,9.393414e+08,30648.6764,0.8922,0.1575,0.1054
6,17656.7221,8.880060e+08,29799.4296,0.8955,0.1558,0.1032
7,18562.4578,9.890121e+08,31448.5632,0.8868,0.1594,0.1051
8,17974.8226,8.862682e+08,29770.2573,0.9003,0.1603,0.1059
9,17842.1570,9.178917e+08,30296.7275,0.8873,0.1544,0.1008


In [31]:
tuned_lightgbm_02.get_params

<bound method LGBMModel.get_params of LGBMRegressor(bagging_fraction=0.8, bagging_freq=2, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.7,
              importance_type='split', learning_rate=0.3, max_depth=-1,
              min_child_samples=66, min_child_weight=0.001, min_split_gain=0.6,
              n_estimators=290, n_jobs=-1, num_leaves=256, objective=None,
              random_state=1838, reg_alpha=0.7, reg_lambda=2, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)>

### **05.04.03. Optimizacion con personalización del espacio muestral de hiperparámetros**

In [32]:
# Espacio muestral de hiperparámetros personalizado
lgbm_params = {'num_leaves': np.arange(10,200,10),
                        'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)],
                        'learning_rate': np.arange(0.1,1,0.1)
              }

In [33]:
tuned_lightgbm_03 = tune_model(model_lightgbm, custom_grid = lgbm_params)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,19082.3692,9.185242e+08,30307.1648,0.8940,0.1652,0.1156
1,18685.6431,8.615378e+08,29351.9646,0.8948,0.1603,0.1130
2,19211.9057,9.139020e+08,30230.8118,0.8907,0.1561,0.1114
3,19060.0664,9.462353e+08,30760.9375,0.8862,0.1581,0.1105
4,18895.3605,8.700903e+08,29497.2933,0.8969,0.1543,0.1092
5,19602.0788,9.709065e+08,31159.3734,0.8885,0.1601,0.1128
6,18522.6546,8.759722e+08,29596.8268,0.8970,0.1525,0.1084
7,19514.2672,9.916816e+08,31490.9759,0.8865,0.1588,0.1108
8,19230.7469,9.136174e+08,30226.1050,0.8972,0.1595,0.1129
9,19250.4171,9.479237e+08,30788.3692,0.8836,0.1584,0.1107


In [34]:
tuned_lightgbm_03.get_params

<bound method LGBMModel.get_params of LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.30000000000000004,
              max_depth=50, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=120,
              objective=None, random_state=1838, reg_alpha=0.0, reg_lambda=0.0,
              silent=True, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0)>

Podemos observar como la métrica evaluada R2 incrementó en los 3 casos ejemplos de optimización. Para ejemplificar las secciones restantes del documento emplearemos el modelo `tuned_lightgbm_02` que fue el que presentó el valor más alto en la métrica priorizada. 
